# Кто первым закончит слово?

Эту игру я впервые увидел в учебнике чтения для первого класса. Википедия описывает очень похожую вариацию игру с потрясающим названием "[антибалда](https://ru.wikipedia.org/wiki/%D0%91%D0%B0%D0%BB%D0%B4%D0%B0_(%D0%B8%D0%B3%D1%80%D0%B0))".

Есть два игрока: назовём их первый и второй. Они по очереди называют буквы, которые записываются последовательно.

Тот, кто первым получит существительное в нормальной форме, побеждает. Имена собственные исключаются.

Если после хода какого-то игрока возникает ситуация, когда невозможно придумать существительное с таким началом, то игрок, сделавший такой ход, объявляется проигравшим.

Пример игры.

Первый: **C**. На доске записано **C**

Второй: **A**. На доске записано **CA**

Первый: **Х**. На доске записано **CАХ**

Если второй сейчас сходит **Г**, то он проиграет, так как нет существительных с первыми буквами **CАХГ**

Второй ходит **А**. На доске записано **CАХА**. Игра не заканчивается, так как Саха - это название республики, имя собственное.

Первый: **Р**. На доске записано **CАХАР**. Слово "сахар" является существительным в нормальной форме. Первый смог составить слово. Первый выиграл!

Кем лучше быть: первый или вторым? С какой буквы лучше начинать игру?

Давайте использовать все 33 буквы алфавита, то есть **Е** и **Ё** - это разные буквы

Для начала загрузим список всех существительных в единственном числе.

Язык — это развивающаяся динамичная структура, слова появляются и исчезают. Получить актуальный список слов можно с помощью регулярно обновляемых словарей, например, на [сайте](https://ruscorpora.ru/) Национального корпуса русского языка.

Чтобы наше исследование было воспроизводимо в любой момент времени, я прилагаю файл `russian_nouns.txt`, в котором собраны интересующие нас существительные. Я находил этот файл на просторах Интернета в 2022 году, сейчас найти первоисточник не удалось.

In [1]:
# Импортируем библиотеки, которые нам понадобятся
import numpy as np
from tqdm import tqdm, trange

In [2]:
# Считаем список существительных
f = open('russian_nouns.txt', 'r', encoding='utf-8')
words = []
while True:
    line = f.readline()
    if not line:
        break
    words.append(line.strip())
f.close()

In [3]:
# Количество слов, которые будут участвовать в игре
len(words)

51301

Исключим слова, которые точно не могут быть получены в игре.

Мы должны исключить все слова, первые несколько букв которых образуют другое слово.

Например, получить слово "поляна" в игре невозможно, так как до этого уже составится "пол" и один из игроков выиграет.

In [4]:
words = sorted(words) # Отсортируем слова в алфавитном порядке
N = len(words) # Число слов в списке
to_delete = np.zeros(len(words), dtype='bool') # Говорим список невозможных в игре слов 

# Все слова, которые в начале содержат слово X, при алфавитном порядке идут подряд следом за словом X
# Для каждого слова X в списке проверяем, содержит ли его следущее слово
for i in trange(N - 1):
    j = i + 1
    while words[j][: len(words[i])] == words[i]: # Проверяем, что первые буквы j-го слова образуют i-ое
        to_delete[j] = True # Если образуют, то ставим флаг на удаление
        j += 1 # и идем проверять следующее слово
        if j == N:
            break
# Теперь удаляем все невозможные слова
words = [words[i] for i in range(len(words)) if not to_delete[i]]
# Получаем, что в игре возможно получить 22597 слов
len(words)

100%|███████████████████████████████████████████████████████████████████████| 51300/51300 [00:00<00:00, 1062027.31it/s]


22597

Теперь генерируем все возможные ситуации, которые могут получиться в игре.

Это все возможные наборы букв, которые являются началом некоторого существительного, которое можно получить по правилам игры.

В том числе стартовая позиция должна быть в этом списке (буквы отсутствуют).

In [5]:
positions = ['']
for word in words:
    for i in range(1, len(word) + 1): # Например, для слова "мопс" добавим "м", "мо", "моп", "мопс"
        positions.append(word[:i])
positions = tuple(sorted(set(positions))) # Убираем повторы через set() и сортируем в алфавитном порядке
N = len(positions) # Число возможных ситуаций в игре
N

97892

В список res будем записывать результат игры, если игра дошла до i-ой позиции.

В список first_move будем записывать, делает ли в i-ой позиции ход первый игрок.

В список is_def будем записывать, определен ли результат игры.

In [6]:
res = np.zeros(N)
first_move = []
for i in range(N):
    if len(positions[i]) % 2 == 0: # На доске четное чисто букв - сейчас ходит первый
        first_move.append(True)
    else: # Иначе ходит второй
        first_move.append(False)
is_def = np.zeros(N)

Полученную игру можно изобразить как [игру в развернутой форме](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D0%B7%D0%B2%D1%91%D1%80%D0%BD%D1%83%D1%82%D0%B0%D1%8F_%D1%84%D0%BE%D1%80%D0%BC%D0%B0_%D0%B8%D0%B3%D1%80%D1%8B). 

Такие игры можно решать "с конца". Сначала смотрим на ситуации, где игра заканчивается, и находим оптимальные действия на последних шагах, где можно составить слово за один ход.

Далее проходимся по всем позициям с неизвестным исходом. Для каждой позиции-буквосочетания X:
- Если мы можем поставить букву так, чтобы получить выигрышную позицию, то позиция X для нас выигрышная
- Если как бы мы не делали ход, получаем проигрышную позицию, то позиция X проигрышная
- Иначе пока недостаточно информации: вернемся к позиции X на следующей итерации

Можно доказать, что для таких игр, как наша (с полной информацией, в развернутой форме), такой подход позволит дать оценку для всех возможных позиций в игре. Этот подход называется [обратной индукцией](https://ru.wikipedia.org/wiki/%D0%9E%D0%B1%D1%80%D0%B0%D1%82%D0%BD%D0%B0%D1%8F_%D0%B8%D0%BD%D0%B4%D1%83%D0%BA%D1%86%D0%B8%D1%8F). 

Реализуем этот подход. Для начала определим, кто выигрывает в узловых точках, где игра заканчивается.

In [7]:
# На наше счастье, и позиции, и слова отсортированы в алфавитном порядке
j = 0
for i in trange(len(words)):
    while words[i] != positions[j]:
        j += 1
    # На этом этапе нашли позицию j, совпадающую с некоторым словом
    # Если должен был ходить первый, но уже из букв получилось слово, то победил второй
    if first_move[j]: 
        res[j] = 2 
    else:
        res[j] = 1
    is_def[j] = 1

100%|███████████████████████████████████████████████████████████████████████| 22597/22597 [00:00<00:00, 1546372.02it/s]


In [8]:
# Число позиций с определенным победителем должно равняться числу слов
assert is_def.sum() == len(words)

Теперь создадим список списков next_pos.

Для каждой i-ой позиции укажем список номеров позиций, в которые можно попасть в один ход из i-ой позиции.

In [9]:
next_pos = []
# Будем активно пользоваться тем, что массив positions отсортирован в алфавитном порядке
for i in trange(N):
    next_i = []
    for j in range(i+1, N): # Все позиции, которые могут получиться из i-ой, следуют после i-ой (т.к. алфавитный порядок)
        # Если длина следующей позиции стала меньше, чем у исходной, то произошла смена первых букв, отвечающих за i-ую позицию
        # Так как позиции расположены в алфавитном порядке, то все позиции с одинаковым началом из k букв идут подряд
        # Значит, если длина j-ой позиции меньше, чем длина i-ой, то при k>j не будет позиций, в которые можно попасть из i
        if len(positions[i]) >  len(positions[j]): 
            break
        # Берем только в случае, если можно попасть за один ход, то есть длины отличаются на 1
        if len(positions[i]) + 1 == len(positions[j]):
            if positions[i] == positions[j][:len(positions[j]) - 1]:
                next_i.append(j)
    next_pos.append(next_i)

100%|█████████████████████████████████████████████████████████████████████████| 97892/97892 [00:01<00:00, 58333.22it/s]


In [10]:
# Пример работы
# Все возможные ходы, которые может сделать второй игрок, если первый сходит "а".
print(positions[1])
after_a = []
for m in next_pos[1]:
    after_a.append(positions[m])
print(after_a)

а
['аб', 'ав', 'аг', 'ад', 'аж', 'аз', 'аи', 'ай', 'ак', 'ал', 'ам', 'ан', 'ао', 'ап', 'ар', 'ас', 'ат', 'ау', 'аф', 'ах', 'ац', 'аш', 'аэ']


Осталось только реализовать алгоритм обратной индукции.

In [11]:
while is_def.sum() < N: # Алгоритм работает, пока результат при правильной игре во всех позициях не будет определен
    for i in range(N):
        if is_def[i] == 0: # Есть смысл проверять только позиции с неясным на данном шаге результатом
            is_win = False # Флаг на победу игрока, который делает текущий ход
            is_lose = True # Флаг на поражение игрока, который делает текущий ход
            if first_move[i]:
                for m in next_pos[i]:
                    if res[m] == 0: # Если ведет в какую-то позицию, где результат не ясен, то не можем сказать, что это проигрыш
                        is_lose = False
                    if res[m] == 1: # Если ведет в какую-то позицию, где победа, то делаем ход в эту позицию и побеждаем
                        is_lose = False
                        is_win = True
                if is_win:
                    res[i] = 1
                    is_def[i] = 1
                if is_lose:
                    res[i] = 2
                    is_def[i] = 1
            else:
                for m in next_pos[i]:
                    if res[m] == 0: # Если ведет в какую-то позицию, где результат не ясен, то не можем сказать, что это проигрыш
                        is_lose = False 
                    if res[m] == 2: # Если ведет в какую-то позицию, где победа, то делаем ход в эту позицию и побеждаем
                        is_lose = False
                        is_win = True
                if is_win:
                    res[i] = 2
                    is_def[i] = 1
                if is_lose:
                    res[i] = 1
                    is_def[i] = 1

Все позиции изучены. 

**Всё готово для ответа на главный вопрос**

Кто же выигрывает в стартовой позиции, когда на доске не написано ни одной буквы?

In [12]:
print("В игре при правильной игре побеждает игрок номер " + str(int(res[0])))

В игре при правильной игре побеждает игрок номер 1


Следующая загадка: С каких букв нужно начинать игру, если хочешь победить?

In [13]:
for m in next_pos[0]:
    print(str(positions[m]) + ' ' + str(int(res[m])))

а 2
б 2
в 2
г 2
д 2
е 2
ж 2
з 2
и 2
й 1
к 2
л 2
м 2
н 2
о 2
п 2
р 2
с 2
т 2
у 2
ф 2
х 2
ц 2
ч 2
ш 2
щ 2
э 2
ю 2
я 2
ё 2


**Результат:** при правильной игре выигрывает первый игрок, причем единственным образом!

Если первым ходом будет записана буква **Й**, то первый может гарантировать себе победу при любых стратегиях второго.

Если первым ходом будет записана НЕ буква **Й**, то второй может гарантировать себе победу при любых стратегиях первого.

**Выигрышная стратегия** выглядит следующим образом.

Первым ходом пишем букву **Й**.

Теперь у нашего соперника остается всего два варианта: **ЙО** и **ЙЕ**. Существительных, которые имели бы в начале две другие буквы, первая из которых **Й**, не существует.

Если соперник написал **О**, то мы сразу выигрываем: **ЙОГ** или **ЙОД**.

Если соперник написал **Е**, то пишем **М**, после чего остаются лишь **ЙЕМЕНЕЦ** и **ЙЕМЕНКА**, в которых нечетное количество букв, а значит мы побеждаем.

Сохраним полученные результаты и создадим игру, которая будет играть в "Кто первый составит слово" с пользователем.

In [14]:
lines = [positions[i] + " " + str(int(res[i])) for i in range(N)]
with open("who_composes_word_solution.txt", 'w+', encoding='utf-8') as output:
    for i in range(N):
        output.write(f'{lines[i]}\n')

## Да начнется игра!

Чтобы игра работала, достаточно запустить только ячейки ниже.

In [15]:
# Импортируем нужные базы
import numpy as np
from tqdm import trange
# Список существительных
f = open('russian_nouns.txt', 'r', encoding='utf-8')
words = []
while True:
    line = f.readline()
    if not line:
        break
    words.append(line.strip())
f.close()
# Файл с позициями и победителем при верной игре
f = open("who_composes_word_solution.txt", "r", encoding='utf-8')
lines = f.readlines()
game_info = [s.split() for s in lines[1:]]
f.close()
# Список позиций, которые могут получиться в игре, и соответствующие им результаты
positions = np.array([""] + [g[0] for g in game_info])
# Список результатов, соответствующих позициям
results = np.array([1] + [int(g[1]) for g in game_info])
# Для каждой i-ой позиции укажем список номеров позиций, в которые можно попасть в один ход из i-ой позиции.
N = len(results)
next_pos = []
# Будем активно пользоваться тем, что массив positions отсортирован в алфавитном порядке
for i in trange(N):
    next_i = []
    for j in range(i+1, N): # Все позиции, которые могут получиться из i-ой, следуют после i-ой (т.к. алфавитный порядок)
        # Если длина следующей позиции стала меньше, чем у исходной, то произошла смена первых букв, отвечающих за i-ую позицию
        # Так как позиции расположены в алфавитном порядке, то все позиции с одинаковым началом из k букв идут подряд
        # Значит, если длина j-ой позиции меньше, чем длина i-ой, то при k>j не будет позиций, в которые можно попасть из i
        if len(positions[i]) >  len(positions[j]): 
            break
        # Берем только в случае, если можно попасть за один ход, то есть длины отличаются на 1
        if len(positions[i]) + 1 == len(positions[j]):
            if positions[i] == positions[j][:len(positions[j]) - 1]:
                next_i.append(j)
    next_pos.append(next_i)

100%|█████████████████████████████████████████████████████████████████████████| 97892/97892 [00:07<00:00, 13952.82it/s]


In [16]:
def make_move(word, num_player, positions=positions, results=results, next_pos=next_pos): # Сделать ход
    ind = np.where(positions == word)[0][0]
    next_ind = next_pos[ind]
    np.random.shuffle(next_ind) # Перемешали, чтобы не ходить всегда в алфавитном порядке
    for n in next_ind:
        if results[n] != num_player:
            return positions[n] # Если можем выиграть, идем по победной стратегии
    ch = np.random.choice(next_ind) # Если не можем выиграть, ходим случайным образом
    return positions[ch]

def give_hint(word, positions=positions, results=results, next_pos=next_pos): # Подсказка
    ind = np.where(positions == word)[0][0]
    next_ind = next_pos[ind]
    hint = []
    for n in next_ind:
        hint.append(positions[n][-1].upper()) # Последняя буква
    print("Доступные буквы: " + ", ".join(sorted(hint)))
    
def input_move(word, positions=positions): # Принять ход игрока
    print('Ваш ход! Введите букву')
    done = False
    while not done:
        s = input().lower() # Делаем букву строчной, если ввели заглавную
        if len(s) != 1:
            print('Вы ввели больше одного символа! Попробуйте еще раз!')
            print('Сейчас на доске написано: ' + word.upper())
        else:
            answer = word + s
            if s == '0':
                give_hint(word)
            elif answer in positions:
                done = True
            else:
                print('Увы, существительных с таким началом нет в русском языке. Попробуйте еще раз!')
                print('Сейчас на доске написано: ' + word.upper())
                print('Чтобы получить подсказку, введите 0')
    return answer

def start_game(positions=positions, results=results, next_pos=next_pos, words=words):
    print("Приветствую на увлекательной лингвистической игре!")
    print("Если хотите ходить первым в игре, введите 1. Если хотите ходить вторым, введите 2.")
    num_player = 0
    while num_player == 0:
        c = input()
        if c == '1':
            num_player = 1
        elif c == '2':
            num_player = 2
        else:
            print("Я не понял ввод. Нужно ввести одну цифру 1 или 2. Попробуйте еще раз!")
    print('Начинаем игру!')
    print('Вводите буквы, чтобы первыми составить слово!')
    curr_pos = ''
    while True:
        if (len(curr_pos) + num_player) % 2 == 0: # Условие на то, что ходит компьютер
            curr_pos = make_move(curr_pos, num_player)
            print("Компьютер сделал ход: " + curr_pos[-1].upper())
        else:
            curr_pos = input_move(curr_pos)
        print('Сейчас на доске написано: ' + curr_pos.upper())
        print()
        if curr_pos in words: #Если было составлено слово, то игра закончилась
            if (len(curr_pos) + num_player) % 2 == 0:
                print("Позравляем!!! Вы победили!")
            else:
                print("Увы, в этот раз компьютер оказался сильнее.")
            break
    print("Сыграйте еще раз!")

In [24]:
start_game()

Приветствую на увлекательной лингвистической игре!
Если хотите ходить первым в игре, введите 1. Если хотите ходить вторым, введите 2.


 1


Начинаем игру!
Вводите буквы, чтобы первыми составить слово!
Ваш ход! Введите букву


 д


Сейчас на доске написано: Д

Компьютер сделал ход: Ь
Сейчас на доске написано: ДЬ

Ваш ход! Введите букву


 я


Сейчас на доске написано: ДЬЯ

Компьютер сделал ход: К
Сейчас на доске написано: ДЬЯК

Увы, в этот раз компьютер оказался сильнее.
Сыграйте еще раз!


In [26]:
start_game()

Приветствую на увлекательной лингвистической игре!
Если хотите ходить первым в игре, введите 1. Если хотите ходить вторым, введите 2.


 я первый!


Я не понял ввод. Нужно ввести одну цифру 1 или 2. Попробуйте еще раз!


 1


Начинаем игру!
Вводите буквы, чтобы первыми составить слово!
Ваш ход! Введите букву


...

Доступные буквы: Н


 н


Сейчас на доске написано: НЯН

Компьютер сделал ход: Ь
Сейчас на доске написано: НЯНЬ

Ваш ход! Введите букву


 к


Сейчас на доске написано: НЯНЬК

Компьютер сделал ход: А
Сейчас на доске написано: НЯНЬКА

Увы, в этот раз компьютер оказался сильнее.
Сыграйте еще раз!


In [27]:
start_game()

Приветствую на увлекательной лингвистической игре!
Если хотите ходить первым в игре, введите 1. Если хотите ходить вторым, введите 2.


 1


Начинаем игру!
Вводите буквы, чтобы первыми составить слово!
Ваш ход! Введите букву


 й


Сейчас на доске написано: Й

Компьютер сделал ход: Е
Сейчас на доске написано: ЙЕ

Ваш ход! Введите букву


...

Сейчас на доске написано: ЙЕМ

Компьютер сделал ход: Е
Сейчас на доске написано: ЙЕМЕ

Ваш ход! Введите букву


 н


Сейчас на доске написано: ЙЕМЕН

Компьютер сделал ход: К
Сейчас на доске написано: ЙЕМЕНК

Ваш ход! Введите букву


 а


Сейчас на доске написано: ЙЕМЕНКА

Позравляем!!! Вы победили!
Сыграйте еще раз!
